In [8]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB  
import subprocess
from sklearn.utils import shuffle
import itertools
from sklearn.ensemble import GradientBoostingClassifier
import sys
inputname="human"
outputname="human_RFH_2RFH_3RFH"
path="data/"

In [2]:
def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError

In [3]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [10]:
"""
used to merge features
"""
RFH_=pd.read_csv(path+"human_RFH_2RFH.csv",header=None,index_col=None)#sys.argv[1])
RFH_=RFH_.values
RFH_=pd.DataFrame(RFH_).astype(float)
PseDNC_=pd.read_csv(path+"human_3RFH.csv",header=None,index_col=None)
PseDNC_=pd.DataFrame(PseDNC_).astype(float)
print "first_file_num:",len(RFH_)
print "first_file_length:",len(RFH_.values[0])
print "second_file_num:",len(PseDNC_)
print "second_file_length:",len(PseDNC_.values[0])
RFH_PseDNC=pd.concat([RFH_,PseDNC_],axis=1)
print "output_file_num:",len(RFH_PseDNC)
print "output_file_len:",len(RFH_PseDNC.values[0])
scaler=MinMaxScaler()
RFH_PseDNC=scaler.fit_transform(np.array(RFH_PseDNC))
pd.DataFrame(RFH_PseDNC).to_csv(path+"human_RFH_2RFH_3RFH.csv",header=None,index=None)

first_file_num: 2260
first_file_length: 364
second_file_num: 2260
second_file_length: 273
output_file_num: 2260
output_file_len: 637


In [ ]:
import csv
import os
import re
"""
    十折交叉验证 + mrmr
"""
classifier="SVM"
datapath =path+outputname+".csv"
outputpath=path+outputname+"_output.csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
row=Y.shape[0]
Y1=Y.reshape(row,1)
#concatenate class and data
full_csv_with_class=np.concatenate([Y1,X],axis=1)
print full_csv_with_class

#print the results of original csv data and final full data
print "the shape of data:"+str(X.shape)
print "the shape of data and class:"+str(full_csv_with_class.shape)

#generating virtual headers
columns=["class"]
columns_numbers=np.arange(full_csv_with_class.shape[1]-1)
columns.extend(columns_numbers)

# Write data into files
csvFile2 = open(outputpath,'w') 
writer = csv.writer(csvFile2)
m = len(full_csv_with_class)
writer.writerow(columns)
for i in range(m):
    writer.writerow(full_csv_with_class[i])
csvFile2.close()

os.system("./mRMR/mrmr -i "+outputpath+" -n "+str(full_csv_with_class.shape[1]-1)+" >mRMR/"+outputname+"_output.mrmrout")
print "mrmr complete "
location_mark=0
final_set=[]
fn=open("mRMR/"+outputname+"_output.mrmrout",'r')
for line in fn.readlines():
    if line.strip() =="":
        location_mark=0
    if location_mark==1 and line.split()[1]!="Fea":
         final_set.append(int(line.split()[1]))
    if re.findall(r"mRMR",line) and re.findall(r"feature",line):
        location_mark=1
print "the set is:",final_set
select_list=[]
precision_copy=0
recall_copy=0
SN_copy=0
SP_copy=0
GM_copy=0
TP_copy=0
TN_copy=0
FP_copy=0
FN_copy=0
ACC_copy=0
F1_Score_copy=0
F_measure_copy=0
MCC_copy=0
pos_copy=0
neg_copy=0
ROC_AUC_area_copy=[]
y_pred_prob_copy=[]
y_pred_copy=[]
for j_num in final_set:
    select_list.append(j_num-1)
    print select_list
    X_2=pd.DataFrame(X).iloc[:,select_list]
    svc = svm.SVC()
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_2, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
    # print clf.best_score_
    gamma=clf.best_params_['gamma']
    print "the best C and gamma are:",C,gamma
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=12)
    ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
    y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=12,method='predict_proba')
    predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    predict_save=np.array(predict_save).T
    pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict_all_pseEIIP.csv',header=None,index=False)
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    print "accuracy: ",ACC
    if ACC_copy<ACC:
        ACC_copy=ACC
        precision_copy=precision
        recall_copy=recall
        SN_copy=SN
        SP_copy=SP
        GM_copy=GM
        TP_copy=TP
        TN_copy=TN
        FP_copy=FP
        FN_copy=FN
        ACC_copy=ACC
        F1_Score_copy=F1_Score
        F_measure_copy=F_measure
        MCC_copy=MCC
        ROC_AUC_area_copy=ROC_AUC_area
        pos_copy=pos
        neg_copy=neg
        y_pred_prob_copy=y_predict_prob
        y_pred_copy=y_predict
savedata=[[[classifier+"C:"+str(C)+"gamma:"+str(gamma),ACC_copy,precision_copy, recall_copy,SN_copy, SP_copy, GM_copy,F_measure_copy,F1_Score_copy,MCC_copy,ROC_AUC_area_copy,
            TP_copy,FN_copy,FP_copy,TN_copy,pos_copy,neg_copy]]]
print savedata
print X_2.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X_2.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'_all_pseEIIP.xls')
    # y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
    # Y=pd.DataFrame(Y)    
    # y_predict_proba=pd.DataFrame(y_predict_proba)
    # y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
    # pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
    # y_predict=pd.DataFrame(y_predict)
    # y_predict_=pd.concat([Y,y_predict],axis=1)
    # pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[ 1.      0.      0.     ...,  0.      1.      0.    ]
 [ 1.      1.      1.     ...,  1.      1.      0.125 ]
 [ 1.      1.      0.     ...,  1.      0.      0.1875]
 ..., 
 [ 0.      1.      1.     ...,  1.      1.      0.0625]
 [ 0.      1.      1.     ...,  0.      1.      0.    ]
 [ 0.      1.      1.     ...,  0.      1.      0.    ]]
the shape of data:(2260, 637)
the shape of data and class:(2260, 638)


In [ ]:
import csv
import os
from sklearn.feature_selection import  f_classif
import re
"""
    十折交叉验证 + f-score
"""
classifier="SVM"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)

F, pval = f_classif(X, Y)
idx = np.argsort(F)
Selected_list_=idx[::-1]

select_list=[]
precision_copy=0
recall_copy=0
SN_copy=0
SP_copy=0
GM_copy=0
TP_copy=0
TN_copy=0
FP_copy=0
FN_copy=0
ACC_copy=0
F1_Score_copy=0
F_measure_copy=0
MCC_copy=0
pos_copy=0
neg_copy=0
ROC_AUC_area_copy=[]
y_pred_prob_copy=[]
y_pred_copy=[]
for j_num in Selected_list_:
    select_list.append(j_num-1)
    print select_list
    X_2=pd.DataFrame(X).iloc[:,select_list]
    svc = svm.SVC()
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_2, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
    # print clf.best_score_
    gamma=clf.best_params_['gamma']
    print "the best C and gamma are:",C,gamma
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=12)
    ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
    y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=12,method='predict_proba')
    predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    predict_save=np.array(predict_save).T
    pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict_output_fscore.csv',header=None,index=False)
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    print "accuracy: ",ACC
    if ACC_copy<ACC:
        ACC_copy=ACC
        precision_copy=precision
        recall_copy=recall
        SN_copy=SN
        SP_copy=SP
        GM_copy=GM
        TP_copy=TP
        TN_copy=TN
        FP_copy=FP
        FN_copy=FN
        ACC_copy=ACC
        F1_Score_copy=F1_Score
        F_measure_copy=F_measure
        MCC_copy=MCC
        ROC_AUC_area_copy=ROC_AUC_area
        pos_copy=pos
        neg_copy=neg
        y_pred_prob_copy=y_predict_prob
        y_pred_copy=y_predict
savedata=[[[classifier+"C:"+str(C)+"gamma:"+str(gamma),ACC_copy,precision_copy, recall_copy,SN_copy, SP_copy, GM_copy,F_measure_copy,F1_Score_copy,MCC_copy,ROC_AUC_area_copy,
            TP_copy,FN_copy,FP_copy,TN_copy,pos_copy,neg_copy]]]
print savedata
print X_2.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X_2.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'output_fscore.xls')
    # y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
    # Y=pd.DataFrame(Y)    
    # y_predict_proba=pd.DataFrame(y_predict_proba)
    # y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
    # pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
    # y_predict=pd.DataFrame(y_predict)
    # y_predict_=pd.concat([Y,y_predict],axis=1)
    # pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

In [18]:
final_out_to_excel=[]
row0 = [u'特征集', u'样本个数', u'分类器', u'Accuracy', u'Precision', u'Recall', u'SN', u'SP',
                u'Gm', u'F_measure', u'F_score', u'MCC', u'ROC曲线面积', u'tp', u'fn', u'fp', u'tn']
final_out_to_excel.append(row0) #above was used to generate xlsx format Excel file
RNA_code='ACGU' 
interval=3 # RNA_code and interval used mix used to generate AAA AAC AAG ...
division_num=10
divided_num=np.float(division_num)# ten fold so the result should divided by ten


# seq=[e.replace('U','A') for e in seq]
# seq=[e.replace('G','C') for e in seq]
seq=pd.read_csv(path+name+'.csv',header=None,index_col=False)
seq=seq.values

positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]



#define variable to save data which will be saved later --- begining
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0
#define variable to save data which will be saved later --- end


#shuffle the data of positive and negative begining
kf = KFold(n_splits=division_num,shuffle=False)  # positive and negative samples will be shuffled 
for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
#shuffle the data of positive and negative end


    #generate train and test data begining
    X_train = np.array(positive_negative_x_train)
    Y_train = list(map(lambda x: 1, xrange(len(X_train) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(X_train) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(positive_negative_y_train)
    Y_test  = list(map(lambda x: 1, xrange(len(X_test) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(X_test) / 2)))
    Y_test.extend(Y2_test )
    Y_test = np.array(Y_test)
    #generate train and test data end

    
    # training model and optimized parameters of C and gamma begining
    svc = svm.SVC(probability=True)
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    # training model and optimized parameters of C and gamma end
    
    
    #print best C and gamma begining
    C=clf.best_params_['C']
    gamma=clf.best_params_['gamma']
    print 'c:',C,'gamma:',gamma
    #print best C and gamma end
    
    
    #getting predict probability and predict label begining
    y_pred_prob=clf.predict_proba(X_test)
    y_pred=clf.predict(X_test)
    #getting predict probability and predict label begining

    #the process of generating  usefule data begining
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)

    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
    #the process of generating  usefule data end
    
#the process of save  data to disk begining   
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
# pd.DataFrame(np.matrix(all_y).T).to_csv('/home02/chenhuangrong/PseEIIP%d.csv'%select_num,header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[str(X_train.shape[1]),"正："+str(pos_all)+'负：'+str(neg_all),'svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all]
final_out_to_excel.append(savedata)

print savedata
pd.DataFrame(final_out_to_excel).to_excel('/home02/chenhuangrong/'+name+'.xlsx',sheet_name="independent_test",index=False,header=False)
#the process of save  data to disk end

c: 32.0 gamma: 1.78179743628
0.83185840708
ACC_all: 0.83185840708
c: 6.34960420787 gamma: 4.0
0.836283185841
ACC_all: 1.66814159292
c: 32.0 gamma: 1.78179743628
0.840707964602
ACC_all: 2.50884955752
c: 6.34960420787 gamma: 4.0
0.840707964602
ACC_all: 3.34955752212
c: 6.34960420787 gamma: 4.0
0.827433628319
ACC_all: 4.17699115044
c: 2.82842712475 gamma: 4.0
0.827433628319
ACC_all: 5.00442477876
c: 32.0 gamma: 0.03125
0.774336283186
ACC_all: 5.77876106195
c: 14.2543794902 gamma: 4.0
0.836283185841
ACC_all: 6.61504424779
c: 2.82842712475 gamma: 4.0
0.849557522124
ACC_all: 7.46460176991
c: 6.34960420787 gamma: 0.353553390593
0.845132743363
ACC_all: 8.30973451327
['20', '\xe6\xad\xa3\xef\xbc\x9a1130.0\xe8\xb4\x9f\xef\xbc\x9a1130.0', 'svmC:6.34960420787gamma:0.353553390593', 0.83097345132743372, 0.929620896040122, 0.7168141592920354, 0.7168141592920354, 0.9451327433628318, 0.8226822953930515, 0.80871204823163612, 0.80871204823163612, 0.68060071085008533, 0.87162581251468407, 810.0, 320.0, 62

In [14]:
"""
    cross validation
"""
classifier="SVM"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
svc = svm.SVC()
parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,28)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,28))}
clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
C=clf.best_params_['C']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
gamma=clf.best_params_['gamma']
y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['SVMC:2.58536929514gamma:0.462937356144', 0.82035398230088497, 0.9031180400890868, 0.7176991150442478, 0.7176991150442478, 0.9230088495575222, 0.8139057896989325, 0.79980276134122286, 0.79980276134122286, 0.65465402754243518, 0.82035398230088497, 811.0, 319.0, 87.0, 1043.0, 1130.0, 1130.0]]]
100


True

In [15]:
"""
    cross validation GBDT
"""
classifier="GBDT"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
GBDT = GradientBoostingClassifier(learning_rate=0.1,min_samples_split=300,min_samples_leaf=20,max_depth=8,max_features='sqrt')
parameters = {'n_estimators':range(10,120,1),'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200),'min_samples_leaf':range(60,101,10)}
clf = GridSearchCV(GBDT, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
n_estimators=clf.best_params_['n_estimators']
max_depth=clf.best_params_['max_depth']
min_samples_split=clf.best_params_['min_samples_split']
min_samples_leaf=clf.best_params_['min_samples_leaf']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
y_predict=cross_val_predict(GradientBoostingClassifier(n_estimators=n_estimators,learning_rate=0.1,min_samples_split=min_samples_split,
                                                       min_samples_leaf=min_samples_leaf,max_depth=max_depth,max_features='sqrt'),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(GradientBoostingClassifier(n_estimators=n_estimators,learning_rate=0.1,min_samples_split=min_samples_split,
                                                       min_samples_leaf=min_samples_leaf,max_depth=max_depth,max_features='sqrt'),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"min_samples_split:"+str(min_samples_split)+"min_samples_leaf:"+str(min_samples_leaf),ACC,precision, recall,SN, SP,
            GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['GBDTn_estimators:68max_depth:3min_samples_split:300min_samples_leaf:100', 0.81327433628318579, 0.890728476821192, 0.7141592920353982, 0.7141592920353982, 0.9123893805309734, 0.8072120874098798, 0.79273084479371325, 0.79273084479371325, 0.63923394144070123, 0.81327433628318579, 807.0, 323.0, 99.0, 1031.0, 1130.0, 1130.0]]]
100


True

In [16]:
"""
    cross validation RF
"""
classifier="RF"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y) 
RF = RandomForestClassifier(min_samples_split=300,min_samples_leaf=20,max_depth=8,max_features='sqrt')
parameters = {'n_estimators':range(10,120,1),'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200),'min_samples_leaf':range(60,101,10)}
clf = GridSearchCV(RF, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
n_estimators=clf.best_params_['n_estimators']
max_depth=clf.best_params_['max_depth']
min_samples_split=clf.best_params_['min_samples_split']
min_samples_leaf=clf.best_params_['min_samples_leaf']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
y_predict=cross_val_predict(RandomForestClassifier(n_estimators=n_estimators,min_samples_split=min_samples_split,
                                                       min_samples_leaf=min_samples_leaf,max_depth=max_depth,max_features='sqrt'),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(RandomForestClassifier(n_estimators=n_estimators,min_samples_split=min_samples_split,
                                                       min_samples_leaf=min_samples_leaf,max_depth=max_depth,max_features='sqrt'),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"min_samples_split:"+str(min_samples_split)+"min_samples_leaf:"+str(min_samples_leaf),ACC,precision, recall,SN, SP,
            GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['RFn_estimators:10max_depth:13min_samples_split:100min_samples_leaf:80', 0.80796460176991147, 0.8849557522123894, 0.7079646017699115, 0.7079646017699115, 0.9079646017699115, 0.8017523294092831, 0.7866273352999017, 0.7866273352999017, 0.62863011097975363, 0.80796460176991147, 800.0, 330.0, 104.0, 1026.0, 1130.0, 1130.0]]]
100


True

In [17]:
"""
    cross validation NB
"""
classifier="NB"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
y_predict=cross_val_predict(GaussianNB(),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(GaussianNB(),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier,ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['NB', 0.81238938053097343, 0.8771367521367521, 0.7265486725663717, 0.7265486725663717, 0.8982300884955752, 0.8078414933980766, 0.7947725072604066, 0.7947725072604066, 0.63419495678799576, 0.81238938053097343, 821.0, 309.0, 115.0, 1015.0, 1130.0, 1130.0]]]
100


True

In [18]:
"""
    cross validation
"""
classifier="LR"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
logisReg = LogisticRegression()
C_range = 10.0 ** np.arange(-4,3,1)
grid_parame = dict(C=C_range)
clf = GridSearchCV(logisReg, grid_parame, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
C=clf.best_params_['C']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_

y_predict=cross_val_predict(LogisticRegression(C=C),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(LogisticRegression(C=C),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"C:"+str(C),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['LRC:1.0', 0.81150442477876106, 0.8430799220272904, 0.7654867256637168, 0.7654867256637168, 0.8575221238938053, 0.8101986193543316, 0.80241187384044521, 0.80241187384044521, 0.62566433426311097, 0.81150442477876106, 865.0, 265.0, 161.0, 969.0, 1130.0, 1130.0]]]
100


True

In [19]:
"""
    cross validation
"""
classifier="KNN"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
knn = KNeighborsClassifier()
#设置k的范围
k_range = list(range(1,10))
leaf_range = list(range(1,2))
weight_options = ['uniform','distance']
algorithm_options = ['auto','ball_tree','kd_tree','brute']
param_gridknn = dict(n_neighbors = k_range,weights = weight_options,algorithm=algorithm_options,leaf_size=leaf_range)
clf = GridSearchCV(knn, param_gridknn, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
n_neighbors=clf.best_params_['n_neighbors']
weights=clf.best_params_['weights']
algorithm=clf.best_params_['algorithm']
leaf_size=clf.best_params_['leaf_size']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_

y_predict=cross_val_predict(KNeighborsClassifier(n_neighbors=n_neighbors,weights=weights,algorithm=algorithm,leaf_size=leaf_size),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(KNeighborsClassifier(n_neighbors=n_neighbors,weights=weights,algorithm=algorithm,leaf_size=leaf_size),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_neighbors:"+str(n_neighbors)+"weights:"+str(weights)+"algorithm:"+str(algorithm)+"leaf_size:"+str(leaf_size),
            ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['KNNn_neighbors:8weights:uniformalgorithm:autoleaf_size:1', 0.80752212389380529, 0.888268156424581, 0.7035398230088495, 0.7035398230088495, 0.911504424778761, 0.8007993891610012, 0.7851851851851851, 0.7851851851851851, 0.62879193285270096, 0.80752212389380529, 795.0, 335.0, 100.0, 1030.0, 1130.0, 1130.0]]]
100


True

In [20]:
"""
    cross validation XGBoost
"""
classifier="XGBoost"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
parameters = [{'n_estimators':range(1,10,1),
                      'max_depth':range(2,10),
                      'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
                      'subsample':[0.75,0.8,0.85,0.9]
                      }]
clf = GridSearchCV(XGBClassifier(), parameters, cv=10, n_jobs=1, scoring='accuracy')
clf.fit(X, Y)
n_estimators=clf.best_params_['n_estimators']
max_depth=clf.best_params_['max_depth']
learning_rate=clf.best_params_['learning_rate']
subsample=clf.best_params_['subsample']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
y_predict=cross_val_predict(XGBClassifier(n_estimators=n_estimators,learning_rate=learning_rate,
                                                       subsample=subsample,max_depth=max_depth),X,Y,cv=10,n_jobs=1)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(XGBClassifier(n_estimators=n_estimators,learning_rate=learning_rate,
                                                       subsample=subsample,max_depth=max_depth),X,Y,cv=10,n_jobs=1,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"learning_rate:"+str(learning_rate)+"subsample:"+str(subsample),ACC,precision, recall,SN, SP,
            GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['XGBoostn_estimators:7max_depth:4learning_rate:0.4subsample:0.9', 0.82168141592920352, 0.8981380065717415, 0.7256637168141593, 0.7256637168141593, 0.9176991150442478, 0.8160520514894094, 0.80274106705824766, 0.80274106705824766, 0.65556419285543932, 0.82168141592920352, 820.0, 310.0, 93.0, 1037.0, 1130.0, 1130.0]]]
100


True